In [1]:
import numpy as np
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, RandomZoom, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.applications import VGG16

In [4]:
X = np.load('<your/path>')

y = np.load('<your/path')

In [ ]:
X.shape, y.shape

In [6]:
X = X / 255

y = y

y = to_categorical(y)

In [ ]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(X.shape[1:]))

for layer in vgg.layers:
    layer.trainable = False

def vgg_model(input_dim, zoom=0, regul=0, drop=0):
  model = Sequential([

  RandomZoom(zoom, input_shape=input_dim),

  vgg,

  GlobalAveragePooling2D(),

  Dense(units=128, activation='relu', kernel_regularizer=regularizers.L2(regul)),
  BatchNormalization(),
  Dropout(drop),

  Dense(units=64, activation='relu', kernel_regularizer=regularizers.L2(regul)),
  BatchNormalization(),
  Dropout(drop),

  Dense(units=32, activation='relu', kernel_regularizer=regularizers.L2(regul)),
  BatchNormalization(),
  Dropout(drop),

  Dense(units=3, activation='softmax')
  ])

  return model

In [ ]:
model = vgg_model(input_dim=(X.shape[1:]),
                  zoom=0.2, regul=10e-6, drop=0.2)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

cb = [EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
      ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=10e-4)]

model.fit(x=X, y=y,
          validation_data=(X, y),
          batch_size=32,
          epochs=75, callbacks=cb)

In [ ]:
# Save
model.save('vgg_model.h5')